![](https://raw.githubusercontent.com/fdannemanndugick/roses2021/main/header.png)

<h1 style="text-align:center; color:#207a38;">ROSES Unit 5 — Mapping (Lecture notebook)</h1>

<div style="text-align:center; font-size:16px">
    <b><a href="https://liam.earth/">Liam Toney</a></b><br>
    <br>
    Geophysical Institute,<br>
    University of Alaska Fairbanks<br>
    <br>
    20 July 2021
</div>

# PyGMT
---
## 1. Basics

First, let's import the PyGMT Python package:

In [ ]:
import pygmt

In PyGMT, every plot or map must start with the creation of a [`pygmt.Figure`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.html#pygmt.Figure) instance. This is similar to the Matplotlib command
```python
fig = plt.figure()
```
which creates a `matplotlib.figure.Figure` instance. (Throughout this lecture, I'll try to connect PyGMT commands to Matplotlib commands.)

In [ ]:
fig = pygmt.Figure()

Now what? All of PyGMT's functionality is described at the PyGMT website, [pygmt.org](https://www.pygmt.org/latest/). It's your one-stop shop for both detailed documentation **and** helpful examples and tutorials. The [API Reference](https://www.pygmt.org/latest/api/index.html) section (API = Application Programming Interface) shows what plotting methods the `pygmt.Figure` class contains. Let's look at the docs for [`Figure.coast`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.coast.html#pygmt.Figure.coast), and use that.

In [ ]:
fig.coast(
    region='g',
    projection='R20/8i',
    shorelines=True,
    water='lightblue',
    land='grey',
    frame=True,
)

...wait. Nothing happened! PyGMT always needs a special closing command, [`Figure.show()`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.show.html#pygmt.Figure.show), to show the figure. It's like using Matplotlib in non-interactive mode, where `plt.show()` or `fig.show()` is required.

In [ ]:
fig.show()

That's better. Our first PyGMT map! We can also show a figure "externally" in a PDF viewer (e.g., in Preview on macOS) with the `method='external'` keyword argument. **Note that this won't work if you're using the cloud-based (JupyterHub) option.**

In [ ]:
import getpass
if getpass.getuser() == 'jovyan':
    print('Can\'t use external viewer on JupyterHub!')
else:
    fig.show(method='external')

You may have noticed that some of the keyword arguments in the above `fig.coast(...)` call are a little opaque. Let's talk about the `region` and `projection` arguments in PyGMT.

* ### The `region` argument

  The `region` argument controls the geographic or Cartesian extent of the figure. If you're familiar with GMT, this is the same as the `-R` flag, as in e.g. `-R45/55/135/145`. (In PyGMT parlance, we call `region` an "alias" for `R`.) In PyGMT, regions are specified as lists of `[xmin, xmax, ymin, ymax]`, e.g. `region=[45, 55, 135, 145]`. However, there's more! GMT (and therefore PyGMT) also supports ISO country code strings as inputs to `region`, e.g. `region='EG'` will produce a map with an extent covering all of Egypt. In the call above, I used another shortcut, `region='g'`, which specifies a **global** domain.

* ### The  `projection` argument

  The `projection` argument controls the map projection. GMT (and therefore PyGMT) supports 31 different map projections, from basic Cartesian axes to [arbitrary vertical perspectives](https://docs.generic-mapping-tools.org/latest/gallery/ex26.html). Again, if you're familiar with GMT, these are specified using the `-J` flag, as in `-JM4i`. In PyGMT, projections codes are still strings, but the `J` is dropped, so an equivalent PyGMT argument would be `projection='M4i'`.

  Projection codes have the general form `<letter_code><reference_coords><scale_or_width>`, with `<reference_coords>` optional for some projections. For example, the code `'M4i'` from the previous paragraph specifies a four-inch-wide   Mercator projection, since `M` is the Mercator code. A more complicated example is `'S-150/90/6i'` — this is a six-inch wide general stereographic projection centered on longitude -150°, latitude 90°. A complete list of projection   codes is given [here](https://docs.generic-mapping-tools.org/latest/proj-codes.html). Note that to use scale instead   of width, you must use a lower-case letter code, e.g. `m` instead of `M` for Mercator. Using the list of projection   codes, can you "translate" the `projection` argument from our first map?
  
Now let's play around with this two important arguments. First, `region`. Try changing the `region`argument around in the below example. Don't forget to try some [ISO codes](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes)!

In [ ]:
fig = pygmt.Figure()
fig.coast(
    region=[-130, -120, 46, 52],
    projection='M4i',
    shorelines=True,
    water='lightblue',
    land='grey',
    frame=True
)
fig.show()

Now let's experiment with the other key parameter, `projection`. Here's the same example as above, but with a different projection:

In [ ]:
fig = pygmt.Figure()
fig.coast(
    region=[-130, -120, 46, 52],
    projection='S-125/90/4i',
    shorelines=True,
    water='lightblue',
    land='grey',
    frame=True,
)
fig.show()

The distortion of different projections is more visible near the poles. Try changing the projection for this map of Alaska:

In [ ]:
fig = pygmt.Figure()
fig.coast(
    region=[171, 231, 50, 72],
    projection='S201/90/8i',
    shorelines=True,
    water='lightblue',
    land='grey',
    frame=0,
    resolution='i',
)
fig.show()

## 2. Some plotting commands

PyGMT's `Figure` class has a number of plotting methods, all of which are documented in the API Reference section of the website. All of these are wrapping GMT commands. Here are a few examples:

### `Figure.plot()`

[`Figure.plot()`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.plot.html#pygmt.Figure.plot) handles the plotting of both line and point data. In GMT, "pen" usually refers to the specification of outlines, such as those of the markers below. `color` specifies the marker fill color. The `style` argument controls the symbol type and the size. There are numerous options, documented in the GMT docs [here](https://docs.generic-mapping-tools.org/latest/plot.html#s). Note that the `-S` is for GMT only and is not needed here, e.g.: `-Sc0.4i` equals `style='c0.4i` and translates to a circle of diameter 0.4 inches.

In [ ]:
from numpy.random import randint

POINT_FILL = 'red'
NUM_PTS = 5

lons = randint(-160, -141, NUM_PTS)
lats = randint(55, 71, NUM_PTS)

# Notice that we're just plotting into the pre-existing Figure instance
fig.plot(
    x=lons,
    y=lats,
    style='a0.2i',
    color=POINT_FILL,
    pen='black',
    label=f'{POINT_FILL}',
)
fig.show()

Legends in GMT are very customizable, but in PyGMT all you must do to create a basic one is add the `label` argument to your `fig.plot()` command and then call [`fig.legend()`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.legend.html#pygmt.Figure.legend). This is similar to Matplotlib's functionality.

In [ ]:
fig.legend()
fig.show()

### `Figure.grdimage()`

[`Figure.grdimage()`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.grdimage.html#pygmt.Figure.grdimage) plots grids, or rasters, on maps. It is similar to Matplotlib's `imshow()`. Importantly, GMT (and therefore PyGMT) can conveniently download and cache a number of useful grids, such as global relief data and NASA satellite imagery. These options are accessed by providing a string beginning with "@" to `Figure.grdimage()` and or grid-related commands. The available datasets with their strings are [here](https://docs.generic-mapping-tools.org/latest/datasets.html). Here are a few examples:

In [ ]:
KWARGS = dict(
    grid='@earth_relief_10m',
    region='g',
    projection='R-130/5i',
    frame=0,
)

fig = pygmt.Figure()
fig.grdimage(**KWARGS)
fig.shift_origin(xshift='3i')  # Shift for next call
fig.grdimage(shading=True, **KWARGS)  # Add illumination!
fig.show()

PyGMT can either use "@" filenames like the above, true filenames for raster files (like GeoTIFFs, etc.), or [`xarray.DataArray`](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html) objects. Below, we create our own grid and feed it to GMT.

In [ ]:
import xarray as xr
import numpy as np

# Define an interesting function of two variables, see:
# https://en.wikipedia.org/wiki/Ackley_function
def ackley(x, y):
    return (
        -20 * np.exp(-0.2 * np.sqrt(0.5 * (x ** 2 + y ** 2)))
        - np.exp(0.5 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y)))
        + np.exp(1)
        + 20
    )

# Create gridded data
INC = 0.05
x = np.arange(-20, 20 + INC, INC)
y = np.arange(-20, 20 + INC, INC)
data = xr.DataArray(ackley(*np.meshgrid(x, y)), coords=(x, y))

fig = pygmt.Figure()
fig.grdimage(
    data,
    frame=True,
    projection='X3i',  # X specifies a simple Cartesian coordinate system
    cmap='inferno',
)
fig.show()

If we instead plot this grid on a geographic projection, GMT will project the grid accordingly. Let's add a colorbar, too. Colorbars are created using [`Figure.colorbar()`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.colorbar.html#pygmt.Figure.colorbar). Try changing the colormap below by giving a different string to `cmap` — GMT colormaps are called color palette tables (CPTs) and they're listed [here](https://docs.generic-mapping-tools.org/6.1/cookbook/cpts.html).

In [ ]:
fig = pygmt.Figure()
fig.coast(region='g', projection='R150/8i', shorelines=True, frame=True)
fig.grdimage(data, cmap='viridis', Q=True)
fig.colorbar(frame='+l"# of kittens"')
fig.show()

## 3. Accessing GMT modules using PyGMT

PyGMT works by 1) wrapping GMT commands (called "modules" in GMT lingo) in a more "Pythonic" shell and 2) facilitating Python data objects to be passed to GMT modules. GMT modules are getting added regularly — but if you don't find the one you need in the API reference, don't fret! You can still call any GMT module using PyGMT using the [`pygmt.clib.Session`](https://www.pygmt.org/latest/api/generated/pygmt.clib.Session.html#pygmt.clib.Session) class's context manager ([what's a context manager?](https://book.pythontips.com/en/latest/context_managers.html)). Below, an example using GMT's [`meca`](https://docs.generic-mapping-tools.org/latest/supplements/seis/meca.html) module, which plots earthquake focal mechanisms:

In [ ]:
fig = pygmt.Figure()

# Create a nice backdrop
fig.coast(
    region=[-153, -147, 60, 62],
    projection='M4i',
    frame=True,
    shorelines=True,
    water='lightblue',
    land='lightbrown',
)

# Use a few context managers to call meca
with pygmt.helpers.GMTTempFile() as temp_file:
    with open(temp_file.name, 'w') as f:
        f.write('-150.02 61.49 48 -3.58 -0.40 3.98 0.46 2.86 0.64 26')
    with pygmt.clib.Session() as session:
        session.call_module('meca', f'{temp_file.name} -Sm0.5i')

fig.show()

...and it just so happens that this module has been recently wrapped! So now we can simply use [`Figure.meca()`](https://www.pygmt.org/latest/api/generated/pygmt.Figure.meca.html#pygmt.Figure.meca). Note the difference in readability:

In [ ]:
fig = pygmt.Figure()

# Create a nice backdrop
fig.coast(
    region=[-153, -147, 60, 62],
    projection='M4i',
    frame=True,
    shorelines=True,
    water='lightblue',
    land='lightbrown',
)

# Call meca
fig.meca(
    longitude=-150.02,
    latitude=61.49,
    depth=48,
    spec=dict(mrr=-3.58, mtt=-0.40, mff=3.98, mrt=0.46, mrf=2.86, mtf=0.64, exponent=26),
    scale='0.5i',
)

fig.show()

Here's another example which calls GMT's [`solar`](https://docs.generic-mapping-tools.org/latest/solar.html) module:

In [ ]:
from obspy import UTCDateTime
from geocoder import osm

now = UTCDateTime()  # Get current UTC time

locate = lambda place : osm(place).latlng[::-1]  # Geocode an arbitrary location string

fig = pygmt.Figure()

pygmt.config(MAP_FRAME_TYPE='plain')  # Use plain map frame

for region, projection, xshift in zip([(-180, 180, 60, 90), 'g'], ['S-150/90/5i', 'R-150/8i'], ['0', '7i']):

    fig.grdimage('@earth_relief_05m', region=region, projection=projection, xshift=xshift)
    fig.coast(shorelines=True, land='lightgray', area_thresh='0/0/1')

    # Use a context manager to call solar
    with pygmt.clib.Session() as session:
        session.call_module('solar', f'-Td+d{now.isoformat()} -Gnavyblue@60')
        
    fig.plot(*locate('Fairbanks, AK'), style='a0.2i', pen=True, color='#f28e2b', frame='a60f15g30')    

# Show the figure externally, if possible
if getpass.getuser() == 'jovyan':
    print('Can\'t use external viewer on JupyterHub!')
else:
    fig.show(method='external')
    
fig.show()

**Note:** Demonstrating the quickly-evolving nature of PyGMT, `solar` has been wrapped in [the latest release of PyGMT, v0.4.0](https://www.pygmt.org/latest/changes.html#release-v0-4-0-2021-06-20). You can view the documentation for the wrapped command [here](https://www.pygmt.org/latest/api/generated/pygmt.Figure.solar.html#pygmt.Figure.solar).

## 4. Resources

* Documentation: [pygmt.org](https://www.pygmt.org/latest/)
* Documentation (development version = master branch on GitHub): [pygmt.org/dev](https://www.pygmt.org/dev/)
* GMT example gallery (for inspiration!): [docs.generic-mapping-tools.org/latest/gallery](https://docs.generic-mapping-tools.org/latest/gallery.html)
* If documentation doesn't have the answer, ask on the GMT forum under the "PyGMT Q&A" category: [forum.generic-mapping-tools.org/c/questions/pygmt-q-a](https://forum.generic-mapping-tools.org/c/questions/pygmt-q-a)
* If you think something isn't working right, file an issue on GitHub: [github.com/GenericMappingTools/pygmt](https://github.com/GenericMappingTools/pygmt)

# cartopy
---
## 1. Basics 

If you've used Matplotlib before, then cartopy should feel familiar — it simply expands Matplotlib's capabilities! Let's start with a simple example similar to the one I showed in my lecture.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

delhi_lon, delhi_lat = 77.23, 28.61

plt.scatter(
    delhi_lon,
    delhi_lat,
    s=120,
    color='green',
    transform=ccrs.Geodetic(),
)

plt.show()

...well, that's not very interesting! Since cartopy is extending Matplotlib, it inherits Matplotlib's default behavior of scaling the axis extents to the data. In this case, this means zooming in on the location of the data point (Delhi, in this case). To regain a global perspective, add `ax.set_global()` right before `plt.show()` above.

Note that the `projection` and `transform` keyword arguments are supplied with objects from the `cartopy.crs` module. These determine the projection of the map and the coordinate reference system of the data to be plotted, respectively. The cartopy folks have a nice write-up on these [here](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html).

Here's a more expanded example which is largely adopted from the cartopy docs [here](https://scitools.org.uk/cartopy/docs/latest/matplotlib/intro.html#adding-data-to-the-map):

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=10))

ax.stock_img()  # Adds an image of topography / bathymetry for quick context

ny_lon, ny_lat = -75, 43

wellington_lon, wellington_lat = 174.8, -41.3

# Plot a line between NY and Delhi, using geodetic transform = gets projected
plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='blue', linewidth=2, marker='o',
         transform=ccrs.Geodetic(),
         )

# Plot same line, but using the same transform as the projection = straight line
plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='black', linestyle='--',
         transform=ccrs.PlateCarree(),
         )

plt.show()

Try plotting the great circle arc between NY and Wellington (coordinates for Wellington are provided). Is it what you expect?

## 2. Resources

The main page of the cartopy website has sections for [Getting started](https://scitools.org.uk/cartopy/docs/latest/#getting-started) and [Getting involved](https://scitools.org.uk/cartopy/docs/latest/#getting-involved). These cover issues such as installation, bug reporting, Q&A, and contributing to the code.